# Conjunto de datos - Cáncer de colon (CRC)

* Introducción
* Exploración del conjunto de datos
* Conjunto de datos 1
* Pre procesado de imágenes
* Mejoras

## Introducción

El conjunto de datos consiste en un grupo de imágenes fueron liberadas bajo [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/) pudiéndose descargar en esta [página](https://zenodo.org/record/53169/export/hx).

Cabe mencionar que la realización de los experimentos fue aprobada por el comité de ética de la Universidad de Heildelberg (Alemania) tal y como mencionan los autores Kather et al. en su [artículo](https://www.nature.com/articles/srep27988?WT.feed_name=subjects_translational-research) y así mismo se siguió la [Declaración de Helsinki](http://www.who.int/bulletin/archives/79%284%29373.pdf).

## Exploración del conjunto de datos

### Conjunto de datos 1

En este conjunto se hallan 8 categorías de tejido, listadas a continuación. En cada una de ellas hay 625 imágenes que **no se solapan entre sí**. Cada imagen tiene un tamaño de 150x150 px (píxeles), que se corresponden con 74$\mathrm{\mu}$m de tejido. En total son 5000 imágenes.

Categorías seleccionadas por los investigadores
1. Epitelio canceroso: Tejido epitelial, una o varias capas de celulas que constituyen revestimiento interno de organos y cavidades asi como la citada mucosa. https://es.wikipedia.org/wiki/Epitelio
2. Estroma simple (de composición homogenea, estroma tumoroso, estroma extra-tumoral y tejido blando): Se trata de la matriz extracelular, es un medio de integracion fisiologico, en el que están embebidas las células, en este caso la composición es homogénea https://es.wikipedia.org/wiki/Estroma_(histolog%C3%ADa) y https://es.wikipedia.org/wiki/Matriz_extracelular
3. Estroma complejo (células tumorosas y/o alguna célula inmune): Como el anterior pero en este caso, se hallan celulas tumorosas y alguna inmune
4. Células inmunes (conglomerado de células inmunes y folículos linfoides submucosos): Foliculo linfoide es una agrupacion de celulas sin organización / estructura, que se encuentra asociado a mucosas. Foliculo linfoide → https://es.wikipedia.org/wiki/Tejido_linfoide_asociado_a_las_mucosas. En el caso de esta investigacion GALT.
5. Material variado (mucosas, hemorrargía, necrosado): necrosado → conjunto de material muerto por patologia
6. Glándulas mucosas normales
7. Tejido adiposo: Tejido conjuntivo cuyas celulas acumulan lipidos en su citoplasma, cumple funciones mecánicas asi como fisiologicas. https://es.wikipedia.org/wiki/Tejido_adiposo
8. Imagen de fondo (no se corresponde con tejido)

### Conjunto de datos 2

Se hallan 10 imágenes grandes, de tamaño 5000x5000 px, que provienen de distinta región con respecto a las imágenes del conjunto anterior. Según los investigadores pueden ser usadas para comprobar distintas combinaciones de descriptores de textura o clasificadores en un entorno real.

Nota 1:

Los autores liberaron los dos conjuntos de imágenes, si bien se realiza la implementación de redes convolucionales con el primero de ellos.

Nota 2:

Al provenir de tan solo 10 pacientes hay que tener en cuenta el sesgo que se comete en esta investigación.

## Conjunto de datos 1

A continuación, se muestra la estructura del conjunto de datos. Las imágenes están ordenadas en subcapetas, habiendo 625 en cada una de ellas. En total 5000.

In [ ]:
PATH_TO_IMG = "input_dataset"
PATH_TO_OUTPUT = "output_dataset"

In [ ]:
img_folder = []
img_labels = []
img_list = []

In [ ]:
import os

img_folder = sorted([folder for folder in os.listdir(PATH_TO_IMG)
                  if os.path.isdir(os.path.join(PATH_TO_IMG, folder))])

print("Categories in the dataset: ",img_folder) # category = label = folder

Nota: La siguiente celda no conviene ejecutarla, es un comando de bash (consola linux)

bash command --> $tree -d img-raw/
bash linux output

img-raw/
├── 01_TUMOR
├── 02_STROMA
├── 03_COMPLEX
├── 04_LYMPHO
├── 05_DEBRIS
├── 06_MUCOSA
├── 07_ADIPOSE
└── 08_EMPTY

8 directories

In [ ]:
for index_folder, category in enumerate(img_folder):
    
    folder = os.path.join(PATH_TO_IMG, category)

    for index_img, img in enumerate(os.listdir(folder)):
        
        if img.endswith(".tif"): # just in case there are other kind of files like .db
            img_list.append(os.path.join(folder, img))
            img_labels.append(img_folder[index_folder])

    print("Category {0:s} has {1:d} images.".format(category, index_img+1))

print()
print("The dataset has: {0:d} images.".format(len(img_labels)))

Cada imagen tiene 150x150 px, por lo que tiene 22500 *características* (features).

Las imágenes histológicas están en formato [TIF](https://en.wikipedia.org/wiki/TIFF).

In [ ]:
# execute once!
# f_img_labels = open(os.path.join(PATH_TO_OUTPUT,'img_labels'),'w')
# f_img_list = open(os.path.join(PATH_TO_OUTPUT,'img_list'),'w')


# for label in img_labels:
#     f_img_labels.write(label+"\n")

# for path in img_list:
#     f_img_list.write(path+"\n")


Se muestra el formato de una imagen aleatoria del conjunto.

In [ ]:
from PIL import Image
import random
import re

# get a random image shape from imgList and its attributes
random_image_path = img_list[random.randint(0, len(img_list))]
random_image = Image.open(random_image_path)
extension, size, mode = random_image.format, random_image.size, random_image.mode

print("Image's relative path: {0:s}".format(random_image_path))
print("Image's category: {0:s}".format(re.split('_\d+',random_image_path.split('/')[1])[0]))
print()
print("Image's metadata")
print("Image's extension: {0:s}. Image's size (width, height): ({1:d}, {2:d}). Image's mode: {3:s}.".\
      format(extension, size[0],size[1],mode))
print()
print("The actual image: ")
print()

random_image

Se comprueba que todas las imágenes del conjunto poseen el mismo formato.

In [ ]:
error = 0

for count, img in enumerate(img_list):
    
    if (Image.open(img).format != 'TIFF' ) or \
    (Image.open(img).size[0] != 150) or \
    (Image.open(img).size[1] != 150) or \
    (Image.open(img).mode != 'RGB'): 
        print("Image %s has got something different: " % img)
        error += 1

if (error == 0): print("Process finished with no errors.")
else: print("There is %d images in the dataset with some distinct image's parameters")

## Pre procesado de imágenes - repasar mejorar textos

Se han creado varias estructuras, y dependiendo de cuál se use se deberá proceder de una forma u otra. En cualquier caso, además se han de tener en cuenta lo siguientes puntos:

* Tamaño de la imagen: Cada arquitectura trabaja con un tamaño de imagen. En la función que toma como argumento la arquitectura de la red convolucional, se especifica este aspecto.

* Canales en la imagen: Tal y como se ha comentado la extensión de las imágenes es TIFF, tiene 3 canales (mode) para los colores rojo, verde y azul, que en inglés se codifican como Red - Green - Blue (RGB) respectivamente. Por defecto en el archivo keras.json (ver jupyter testing/system) se especifica el *"formato": "canales al final"*, en inglés *"image_data_format": "channels_last"*. Por lo que, las imágenes se deberán pasar de esta forma <lote, altura, ancho, canales>, en inglés <batch, height, width, channels>.

* Estructura del conjunto de datos: Con el atributo lotes del apartado anterior, se especifica el número de imágenes (lote) 

* Aumento del conjunto de imágenes (data augmentation): 

* Procedimiento - Pipeline: 



In [ ]:
# pre processing --> esto es un ejemplo porque se hara con funciones implementadas en keras

def img_preprocessing(img, arch):
    
    # https://python-pillow.org/pillow-perf/
    aux = Image.open(img)
    
    # input for VGG16 = VGG19 = resnet50 --> 224 x 224px (width x height)
    
    if(arch=="inception"):
#         return aux.resize((width,height),resample = Image.BICUBIC)
        return aux.resize((299,299),resample = Image.LANCZOS)
    else:
#         return aux.resize((224,224),resample = Image.BICUBIC)
        return aux.resize((224,244),resample = Image.LANCZOS)

Comprobamos el funcionamiento de la función

In [ ]:
example_before = Image.open(random_image_path)
example_before 

In [ ]:
example_after = img_preprocessing(random_image_path, "VGG16")
example_after

# data augmentation with keras

In [1]:
%run '../system/functions.ipynb' # import util functions

In [2]:
from time import time # execution time no the best accurate but the easiest sometimes does not work

In [13]:
# definidas antes

# PATH_TO_IMG = "input_dataset"
# PATH_TO_OUTPUT = "output_dataset"

# img_labels & img_list - completed path to file
PATH_TO_LABELS_FILE = "output_dataset/img_labels"
PATH_TO_PATHS_FILE = "output_dataset/img_list"

# output convnet files path + feed
PATH_TO_OUTPUT = "output_convnet"

PATH_TO_DATA_AUGMENTATION = "output_data_augmentation"

In [4]:
img_labels = []
img_list = []

'''
para eliminar el retorno de carro --> [:-1]
para encontrar el path correcto hay que transformar 
la ruta puesto que este fichero y desde el que se crea tienen rutas distintas
'''
# input_dataset/01_TUMOR/2642_CRC-Prim-HE-07_025.tif_Row_1351_Col_601.tif

with open(PATH_TO_LABELS_FILE,'r') as f_img_labels:
    for line in f_img_labels:
        img_labels.append(line[:-1]) 

with open(PATH_TO_PATHS_FILE,'r') as f_img_list:
    for line in f_img_list:
        img_list.append("../data/"+line[:-1]) # para eliminar el retorno de carro

In [5]:
import numpy as np

img_labels_arr = np.array(img_labels)
img_list_arr = np.array(img_list)

In [6]:
# proporciones elegidas 60-20-20 // entrenamiento validacion y test
TRAIN = 0.6
VAL = 0.2

# index selection
index_selection = np.arange(len(img_list)) # tamaño de la lista

np.random.shuffle(index_selection) # ahora en index_selection tenemos aleatorizado el vector

# ahora tenemos aleatorizados los vectores de la misma manera (pareados)

img_labels_arr_rand = img_labels_arr[index_selection]
img_list_arr_rand = img_list_arr[index_selection]

slice_1 = int(TRAIN*len(img_list)) # [0:2999] -> 3000 images to train
slice_2 = int(slice_1+VAL*len(img_list))  # [3000:3999] -> 1000 images to validate, another 1k to test

# print(slice_1)
# print(slice_2)

# muestras de entrenamiento

labels_train = img_labels_arr_rand[:slice_1] 
img_list_train = img_list_arr_rand[:slice_1]

# muestras validacion

labels_val = img_labels_arr_rand[int(slice_1):slice_2] # 2999 +1 to 2999+1+1000
img_list_val = img_list_arr_rand[int(slice_1):slice_2]

# muestras test

labels_test = img_labels_arr_rand[slice_2:]
img_list_test = img_list_arr_rand[slice_2:]

In [7]:
print("tamaño muestra entrenamiento")
print(labels_train.shape[0])
print(img_list_train.shape[0])
print()
print("tamaño muestra validacion")
print(labels_val.shape[0])
print(img_list_val.shape[0])
print()
print("tamaño muestra test")
print(labels_test.shape[0])
print(img_list_test.shape[0])
print()

tamaño muestra entrenamiento
3000
3000

tamaño muestra validacion
1000
1000

tamaño muestra test
1000
1000



In [ ]:
# print(labels_train)
# print(img_list_train)

In [ ]:
# print(labels_val)
# print(img_list_val)

In [ ]:
# print(labels_test)
# print(img_list_test)

Data augmentation, declaramos una funcion, le pasamos como argumento el listado de fotos de train, validation y test y guardamos esas fotos en un directorio creado a tal efecto

In [15]:
# data augmentation -->
# image example data augmentation with keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

# generador de aleatoriedad revisar argumentos estos son unos pocos
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


# dataset_to_aug is a list, train, validation or test // type_of_dataset train val or test
def data_augmentation(dataset_to_aug,labels_to_aug,type_of_dataset):
    for image in dataset_to_aug:
        img = load_img(image, target_size=(224, 224), interpolation='lanczos')
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape) # (1,224,224,3)
        
        #data augmentation
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir=type_of_dataset, save_format='jpeg'):
            i += 1
            if i > 20:
                break  # otherwise the generator would loop indefinitely


# salvar las fotos en estructura de directorios

In [14]:
# train_dir = os.path.join(PATH_TO_DATA_AUGMENTATION,'train')
# val_dir = os.path.join(PATH_TO_DATA_AUGMENTATION,'validation')
# test_dir = os.path.join(PATH_TO_DATA_AUGMENTATION,'test')

# data_augmentation(img_list_test,labels_test,'test') # function to save img transformed

In [17]:
# dataset_to_aug is a list, train, validation or test // type_of_dataset train val or test


    img = load_img(image, target_size=(224, 224), interpolation='lanczos')
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape) # (1,224,224,3)


# salvar las fotos en estructura de directorios

In [ ]:
# image example data augmentation with keras
from keras.preprocessing.image import ImageDataGenerator

newDataSet = ImageDataGenerator(rotation_range=40,width_shift_range=0.2,\
                                height_shift_range=0.2, rescale=1./255, shear_range=0.2,\
                                zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

## Mejoras

Relativas a la posible mejora de los resultados analíticos:

* aumento de datos (data augmentation) con Keras

Relativas a la mejora de los códigos Python:

* mejora de código en general y estructuras, usando otras librerias más eficientes
* mejora del análisis del conjunto de datos (comprobación de los formatos de las imágenes etc.)
* control de errores en cada parte del proceso